In [1]:
import pandas as pd    

# Read in training data as json 
jsonObj = pd.read_json(path_or_buf="../data/train.jsonl", lines=True)

In [93]:
counter = 0
index_list = []

# Identify abstractive spoiler and save their index in a list
for i, j in jsonObj.iterrows():
    string = ""
    for k in j["targetParagraphs"]:
        string += k
    for l in j["spoiler"]:
        if l not in string:
            index_list.append(i)
            counter +=1
            
print(counter)

152


In [94]:
# Drop abstractive spoilers from training data

jsonObj = jsonObj.drop(index=index_list)

In [7]:
list_of_spoiler_dicts = []
for i,j in jsonObj.iterrows():
    
    # Create dictionary for every spoiler
    spoiler_dict = {}
    
    # Add Key-Value pair for title to the dictionary
    spoiler_dict["title"] = j["targetTitle"]
    
    # Add Key-Value pair in the dictionary with a list for the multiple spoilers
    spoiler_dict["paragraphs"] = []
    
    # Dictionary with the metadata for the various spoilers in the spoiler text
    paragraph_dict = {}
    
    # Add UUID of the spoiler as id to the corresponding paragraph
    paragraph_dict["id"] = j["uuid"]
    
    # Create empty string to append all paragraphs and recreated the full context
    context = ""
    
    # Recalculate the string position of the spoiler starts due to the recreating of the full context
    offset = 0
    counter = 0
    
    # Recreating the full context from the single paragraphs
    for k in j["targetParagraphs"]:
        context += k + " "

    
    list_ans = []        
    
    dict_ans = {}
    
    # Add question/postText to the dictionary, that contains the answer
    dict_ans["question"] = j["postText"][0]
    
    # Add UUID of the spoiler as id to the corresponding paragraph
    dict_ans["id"] = j["uuid"]
    
    list_ans_2 = []
    
    for l in range(len(j["spoiler"])):
        
        # Create dictionary for the answer metadata 
        dict_ans_2 = {}
        
        # Add the spoiler text to the "text" key in the dictionary
        dict_ans_2["text"] = j["spoiler"][l]
        
        # Create variables to recreate the answer start position
        offset = 0
        counter = 0
    
        # Recalculate the answer start position in the full context
        for k in j["targetParagraphs"]:
            if counter == j["spoilerPositions"][l][0][0]:
                break
            offset += len(k) + 1
            counter += 1
            
        
        # Reformating the offset for the query start regarding the amount of whitespaces, that where added because of the recreation of the context
        if len(j["spoilerPositions"][l]) == 2:
            if j["spoilerPositions"][l][0][0] == j["spoilerPositions"][l][1][0]:
               
                if context[j["spoilerPositions"][l][0][1] + offset : j["spoilerPositions"][l][0][1] + offset + len(j["spoiler"][l])].startswith(" "):
                    dict_ans_2["answer_start"] = j["spoilerPositions"][l][0][1] + offset + 1
                elif context[j["spoilerPositions"][l][0][1] + offset : j["spoilerPositions"][l][0][1] + offset + len(j["spoiler"][l])].endswith(" "):
                    dict_ans_2["answer_start"] = j["spoilerPositions"][l][0][1] + offset + 2
                else:
                    dict_ans_2["answer_start"] = j["spoilerPositions"][l][0][1] + offset
            else:
                dict_ans_2["answer_start"] = j["spoilerPositions"][l][0][1] - 1

        elif len(j["spoilerPositions"][l]) == 1:
            dict_ans_2["answer_start"] = j["spoilerPositions"][l][0][1] 

        list_ans_2.append(dict_ans_2)
        
    
    dict_ans["answers"] = list_ans_2
    
    # Add the key-value pair for is_impossible. Always False, because abstrative spoiler were excluded beforehand 
    dict_ans["is_impossible"] = False

    
    list_ans.append(dict_ans)
    
    # Creating dictionary that contains the context and the questions with the corresponding metadata
    dict_qas_con = {}
    
    # Add questions and corresponding metadata like (answer_start) to the dictionary key "qas"
    dict_qas_con["qas"] = list_ans
    
    # Adding the full context to the dictionary
    dict_qas_con["context"] = context
    spoiler_dict["paragraphs"].append(dict_qas_con)
   

    list_of_spoiler_dicts.append(spoiler_dict)

dict_final = {}

# Add the version information to the json 
dict_final["version"] = "v2.0"

# Manually deleted spoilers, that cause issues

##del list_of_spoiler_dicts[1767]
##del list_of_spoiler_dicts[1703]
#del list_of_spoiler_dicts[1633]
#del list_of_spoiler_dicts[1632]
#del list_of_spoiler_dicts[1631]
#del list_of_spoiler_dicts[1630]
#del list_of_spoiler_dicts[1629]
#del list_of_spoiler_dicts[1628]
#del list_of_spoiler_dicts[1600:1700]
dict_final["data"] = list_of_spoiler_dicts#[1628:1629]
import json

# Save Squad2.0 formatted training data as json 
with open('../data/train_squad_format_new_3.json', 'w') as fp:
    json.dump(dict_final, fp)

1367


In [5]:
jsonObj

,uuid,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags
0,0af11f6b-c889-4520-9372-66ba25cb7657,532quh,"[Wes Welker Wanted Dinner With Tom Brady, But ...",reddit,[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,","[http://pixel.wp.com/b.gif?v=noscript, http://...",http://nesn.com/2016/09/wes-welker-wanted-dinn...,"{'source': 'anonymized', 'humanSpoiler': 'They...",[how about that morning we go throw?],"[[[3, 151], [3, 186]]]",[passage]
1,b1a1f63d-8853-4a11-89e8-6b2952a393ec,411701128456593408,[NASA sets date for full recovery of ozone hole],Twitter,[2070 is shaping up to be a great year for Mot...,Hole In Ozone Layer Expected To Make Full Reco...,2070 is shaping up to be a great year for Moth...,"ozone layer,ozone hole determined by weather,M...",[http://s.m.huffpost.com/assets/Logo_Huffingto...,http://huff.to/1cH672Z,"{'source': 'anonymized', 'humanSpoiler': '2070...",[2070],"[[[0, 0], [0, 4]]]",[phrase]
2,008b7b19-0445-4e16-8f9e-075b73f80ca4,380537005123190784,[This is what makes employees happy -- and it'...,Twitter,"[Despite common belief, money isn't the key to...",Intellectual Stimulation Trumps Money For Empl...,By: Chad Brooks \r\nPublished: 09/18/2013 06:4...,"employee happiness money,employee happiness in...",[http://i.huffpost.com/gen/1359674/images/o-HA...,http://huff.to/1epfeaw,"{'source': 'anonymized', 'humanSpoiler': 'Inte...",[intellectual stimulation],"[[[1, 186], [1, 210]]]",[phrase]
3,31ecf93c-3e21-4c80-949b-aa549a046b93,844567852531286016,[Passion is overrated — 7 work habits you need...,Twitter,"[It’s common wisdom. Near gospel really, and n...","‘Follow your passion’ is wrong, here are 7 hab...",There's a lot more to work that loving your job,"business, work-life, careers",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[Purpose connects us to something bigger and i...,"[[[11, 25], [11, 101]], [[17, 56], [17, 85]], ...",[multi]
4,31b108a3-c828-421a-a4b9-cf651e9ac859,814186311573766144,[The perfect way to cook rice so that it's per...,Twitter,"[Boiling rice may seem simple, but there is a ...",Revealed: The perfect way to cook rice so that...,The question 'How does one cook rice properly?...,"Quora,users,share,perfect,way,cook,rice",None,None,"{'source': 'anonymized', 'humanSpoiler': None,...",[in a rice cooker],"[[[5, 60], [5, 76]]]",[phrase]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3195,92578045-699f-4957-a3c5-cff2c3874dae,591979258442027008,[Has Facebook's video explosion completely sha...,Twitter,"[A long time ago in a galaxy far, far away...W...","Facebook Video Surging, But YouTube Still Offe...",.,"Facebook,web video,web video ads,YouTube",[http://djcs-prod.s3.amazonaws.com/wsjfrontpag...,http://on.wsj.com/1GfyUqz,"{'source': 'anonymized', 'humanSpoiler': 'No.'...",[it hasn’t necessarily taken the wind out of Y...,"[[[7, 50], [7, 118]]]",[passage]
3196,51682121-df0b-4289-a95f-e1bc3d181306,881531941974661_902284433232745,[Cop Is Eating At A Chili's When Teen Hands Hi...,Facebook,"[The Kansas City, Kansas Police Department are...",Cop is eating at Chili's when teen hands him f...,None,None,[https://sbly-web-prod-shareably.netdna-ssl.co...,http://shareably.net/officer-receives-touching...,"{'source': 'anonymized', 'humanSpoiler': 'The ...","[It read, ""Thanks for keeping us safe.""]","[[[0, 317], [0, 355]]]",[passage]
3197,9c45ca67-38c4-47b4-aa0d-48434bae09fc,837356193576333314,[5 popular myths about visible signs of aging ...,Twitter,"[Obama looks decades younger already, but what...",5 popular myths about visible signs of aging t...,Dozens of anti-wrinkle products and creams are...,,None,https://www.businessinsider.in/5-popular-myths...,"{'source': 'anonymized', 'humanSpoiler': None,...",[1. Anti-wrinkle creams will er

In [57]:
import json

# Read in validation dataset
with open('../data/validation.jsonl', 'r', encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

In [58]:
# Transform validation dataset dictionary into pandas dataframe 
df = pd.DataFrame(data)

In [84]:
counter = 0
index_list = []

# Identify abstractive spoiler and save their index in a list
for i, j in df.iterrows():
    string = ""
    for k in j["targetParagraphs"]:
        string += k
    for l in j["spoiler"]:
        if l not in string:
            index_list.append(i)
            counter +=1
            
print(counter)

30


In [85]:
# Drop abstractive spoilers from validation data

df = df.drop(index=index_list)

In [88]:
# Transforming the validation dataset into Squad2.0 format is similar to the transformation of the training dataset above
# Therefore the documentation is also the same

list_of_spoiler_dicts = []
for i,j in df.iterrows():

    spoiler_dict = {}
    

    spoiler_dict["title"] = j["targetTitle"]
    

    spoiler_dict["paragraphs"] = []
    paragraph_dict = {}
    
    paragraph_dict["id"] = j["uuid"]
    
    context = ""
    offset = 0
    counter = 0
    for k in j["targetParagraphs"]:
        context += k + " "

    
    list_ans = []        
    dict_ans = {}
    dict_ans["question"] = j["postText"][0]
    dict_ans["id"] = j["uuid"]
    list_ans_2 = []
    for l in range(len(j["spoiler"])):
        dict_ans_2 = {}
        dict_ans_2["text"] = j["spoiler"][l]
        offset = 0
        counter = 0
       
        for k in j["targetParagraphs"]:
            if counter == j["spoilerPositions"][l][0][0]:
                break
            offset += len(k) + 1
            counter += 1
            
     
        if len(j["spoilerPositions"][l]) == 2:
            if j["spoilerPositions"][l][0][0] == j["spoilerPositions"][l][1][0]:
               
                if context[j["spoilerPositions"][l][0][1] + offset : j["spoilerPositions"][l][0][1] + offset + len(j["spoiler"][l])].startswith(" "):
                    dict_ans_2["answer_start"] = j["spoilerPositions"][l][0][1] + offset + 1
                elif context[j["spoilerPositions"][l][0][1] + offset : j["spoilerPositions"][l][0][1] + offset + len(j["spoiler"][l])].endswith(" "):
                    dict_ans_2["answer_start"] = j["spoilerPositions"][l][0][1] + offset + 2
                else:
                    dict_ans_2["answer_start"] = j["spoilerPositions"][l][0][1] + offset
            else:
                dict_ans_2["answer_start"] = j["spoilerPositions"][l][0][1] - 1

        elif len(j["spoilerPositions"][l]) == 1:
            dict_ans_2["answer_start"] = j["spoilerPositions"][l][0][1] 

     
        list_ans_2.append(dict_ans_2)
        
    
    dict_ans["answers"] = list_ans_2
    dict_ans["is_impossible"] = False

    
    list_ans.append(dict_ans)
    dict_qas_con = {}
    dict_qas_con["qas"] = list_ans
    
    
    
    
    dict_qas_con["context"] = context
    spoiler_dict["paragraphs"].append(dict_qas_con)
   

    list_of_spoiler_dicts.append(spoiler_dict)

dict_final = {}
dict_final["version"] = "v2.0"

#del list_of_spoiler_dicts[1600:1700]
#del list_of_spoiler_dicts[731]
del list_of_spoiler_dicts[650:750]
dict_final["data"] = list_of_spoiler_dicts#[:750]
import json

with open('../data/val_squad_format_new.json', 'w') as fp:
    json.dump(dict_final, fp)